# Section 14: Drawing Insight From Model

### Power Insights from your CAP Curve
- **Likelihood Score**
    - The score that your customers will churn in this case. This can help you provide offers and segment customers based on their score.
- ** Budgent Constraint**
    -  Given the amount of money we need to spend, we can use the CAP Curve to find the apporpriate customers to reach out
- **Efficiency**
    - Sometimes you do not have a budgent constraint but it can help you find the optimal customers
    
### Coefficients of a Logistic Regression
- Finding ways to understand the way of regression
- ODDS Ration => Coefficients => Practical Application

### Odds Ratio
- Losing to Winning Odds
- - <img src="./images/AZ_14_1.png" alt="Drawing" style="width: 200px;"/>
    - Looking at the example, we need to think of it the lose and gain.
    - If you put 1 dollar in the horse, you either could lose 1 dollar or gain 10 dollars (1:10)
    - The Losing to Winning Odds are 10 to 1 (10:1) Will win only 1 out of 11 times
    - An 11 Dollars odds means a LOSING to WINNING Ratio of 10:1
    - Prob. of winning (1/11) = 9%
    - <img src="./images/AZ_14_2.png" alt="Drawing" style="width: 200px;"/>
    
### Odds Ratio Coefficients
- Wins: Lose Odds = (p/1-p)
- CONCLUSION: Increasing an independent variable x by 1 unit will increase the odds by a multiplicative factor of $e^{b}$
- The coefficients can tell us information. For example, using the log regression, we can use $e^{b}$ where b is the coefficients. This will get us the probability of the outcome
- Remeber the Odds Ratio is not a percentage. Instead, say we have an odd ratio of 2, that means that the prob. of that variable will be 2 times greater than without it. 
- The main thing you should care about the extremes from both sides. If its greater, than 1.2, you should care about it. If its less than 0.8, you should also care about it. 
- Using the information from both sides, we can get the information in the Excel. For example, if the odds ratio is 0.3, then we must be aware that this addtional var has is 70% less likely than with it. 

In [10]:
import pandas as pd
import numpy as np

from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

from scipy import stats
from statsmodels.api import add_constant
import statsmodels.formula.api as sm

PATH = '/Users/alexguanga/Downloads/'


In [2]:
df_test = pd.read_csv(PATH+"Churn-Modelling-Test-Data.csv")
df_train = pd.read_csv(PATH+"Churn-Modelling.csv")

In [3]:
gender_dummy = pd.get_dummies(df_train['Gender'])
df_train = pd.concat([df_train, gender_dummy], axis=1)
df_train.drop(['Gender', 'Male'], axis=1, inplace=True) 

geo_dummy = pd.get_dummies(df_train['Geography'])
df_train = pd.concat([df_train, geo_dummy], axis=1)
df_train.drop(['Geography', 'France'], axis=1, inplace=True) 

df_train['Log_Balance'] = np.log(df_train['Balance'] + 1)

y = df_train['Exited']

df_train.head()

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Female,Germany,Spain,Log_Balance
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,0.000000
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,1,0,1,11.336294
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,11.980813
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,0.000000
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,1,0,1,11.740155


In [8]:
features_2 = df_train[['CreditScore', 'Age', 'NumOfProducts', 'IsActiveMember', 
                       'Female', 'Germany', 'Log_Balance']]

In [11]:
Xmat = add_constant(features_2)
logit = sm.Logit(y, Xmat)

# fit the model
model = logit.fit()

Optimization terminated successfully.
         Current function value: 0.428401
         Iterations 6


In [12]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Exited   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9992
Method:                           MLE   Df Model:                            7
Date:                Fri, 12 Jan 2018   Pseudo R-squ.:                  0.1525
Time:                        09:57:39   Log-Likelihood:                -4284.0
converged:                       True   LL-Null:                       -5054.9
                                        LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -3.9924      0.233    -17.163      0.000      -4.448      -3.536
CreditScore       -0.0007      0.000     -2.407      0.016      -0.001      -0.000
Age                0.0726      0.003     28.220      0.000       0.068       0.078
NumOfProducts     -0.0955      0.048     -2.010      0.044      -0.189      -0.002
IsActiveMember    -1.0725      0.058    -18.621      0.000      -1.185      -0.960
Female             0.5283      0.054      9.704      0.000       0.422       0.635
Germany            0.7463      0.065     11.475      0.000       0.619       0.874
Log_Balance        0.0300      0.006      4.947      0.000       0.018       0.042
==================================================================================
"""

In [29]:
for key, param in model.params.items():
    if key != 'const':
        print(np.exp(param))

0.999325847119
1.07534391351
0.908923782303
0.342142738735
1.69604872548
2.10918696742
1.03043382483
